## Script for all analyses relating to leatherback KDE analyses

  #### 1. Data preparation
     a. Upload the data file
     b. Delete surplus fields
     c. Select location points within a radius
     d. Export radius selection to new feature layer
      
  #### 2. KDE generation
  #### 3. Quantiles

### 1. Data preparation

Upload the data file. Use the file: "[turtle_name]-[turtleID#]-1-FastGPS.csv"

Data Management Tool:  XYTableToPoint

In [ ]:
arcpy.management.XYTableToPoint(
    in_table="Queen Haevo-237820-1-FastGPS.csv",
    out_feature_class=r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\QueenHaevo237820",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)


**Delete surplus fields from base Feature Layer generated from the "[turtle_name]-[turtleID#]-1-FastGPS.csv" files**

Data Management Tool: _Delete Field_ (use Keep Fields method for required fields)

In [ ]:
arcpy.management.DeleteField(
    in_table="QueenHaevo237820", #replace 
    drop_field="Name;Day;Time;Hauled_Out;Latitude;Longitude",
    method="KEEP_FIELDS")

**Select all location points within a given radius around a nesting beach**

Data Management Tool: _Select Layer by Location_

In [ ]:
arcpy.management.SelectLayerByLocation(
    in_layer="QueenHaevo237820",
    overlap_type="INTERSECT",
    select_features="HaevoBeach_reference",
    search_distance="100 Kilometers",
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT")

Export selected points from previous step to a new Feature Layer

Conversion Tool: _ExportFeatures_

In [ ]:
arcpy.conversion.ExportFeatures(
    in_features="QueenHaevo237820",
    out_features=r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\QueenHaevo237820_100km",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='Name "Name" true true false 8000 Text 0 0,First,#,QueenHaevo237820,Name,0,8000;Day "Day" true true false 8000 Text 0 0,First,#,QueenHaevo237820,Day,0,8000;Time "Time" true true false 8000 Text 0 0,First,#,QueenHaevo237820,Time,0,8000;Hauled_Out "Hauled Out" true true false 4 Long 0 0,First,#,QueenHaevo237820,Hauled_Out,-1,-1;Latitude "Latitude" true true false 8 Double 0 0,First,#,QueenHaevo237820,Latitude,-1,-1;Longitude "Longitude" true true false 8 Double 0 0,First,#,QueenHaevo237820,Longitude,-1,-1',
    sort_field=None)

### 2. Generate Kernel Densities (KDE)

Spatial Analyst Tool: _KernelDensity_

In [ ]:
out_raster = arcpy.sa.KernelDensity(
    in_features="QueenHaevo237820_100km",
    population_field="NONE",
    cell_size=0.0005,
    search_radius=28000,
    area_unit_scale_factor="SQUARE_KILOMETERS",
    out_cell_values="DENSITIES",
    method="GEODESIC",
    in_barriers="Solomons_bound_100kmSimp"
)
out_raster.save(r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\QueenHaevoKDE_0005_28000")

### 3. Creating Quantiles at 50 & 95%

Extract kernel density values for location points

Spatial Analyst Tool: _ExtractValuesToPoints_

In [ ]:
arcpy.sa.ExtractValuesToPoints(
    in_point_features="QueenHaevo237820_100km",
    in_raster="QueenHaevoKDE_0005_28000",
    out_point_features=r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\KDvalues_237820",
    interpolate_values="NONE",
    add_attributes="VALUE_ONLY")

Reclassify to generate 50% quantile
Spatial Analyst Tool: Reclassify

In [ ]:
out_raster = arcpy.sa.Reclassify(
    in_raster="QueenHaevoKDE_0005_28000",
    reclass_field="VALUE",
    remap="0 0.194193 NODATA;0.194193 0.318456 50",
    missing_values="DATA"
)
out_raster.save(r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\HQ237820_50")

Reclassify to generate 95% quantile
Spatial Analyst Tool: Reclassify

In [ ]:
out_raster = arcpy.sa.Reclassify(
    in_raster="QueenHaevoKDE_0005_28000",
    reclass_field="VALUE",
    remap="0 0.043070 NODATA;0.043070 0.318456 95",
    missing_values="DATA"
)
out_raster.save(r"\\Mac\Home\Documents\ArcGIS\Projects\Leatherback_KDEs\Leatherback_KDEs.gdb\HQ237820_95")